In [58]:
# Example notebook writing OpenPTV files for Paraview

In [59]:
import numpy as np
import scipy
from flowtracks.io import trajectories_ptvis

In [60]:
dataset = trajectories_ptvis('/home/user/Downloads/ptv_is_for_paraview/ptv_is.%d',xuap=False)

In [61]:
len(dataset)

7362

In [62]:
dataset[0].trajid(), dataset[0].pos(), dataset[0].velocity(), dataset[0].time()

(257,
 array([[ 0.016665,  0.026022, -0.009375],
        [ 0.016737,  0.025983, -0.009735]]),
 array([[ 7.2e-05, -3.9e-05, -3.6e-04],
        [ 0.0e+00,  0.0e+00,  0.0e+00]]),
 array([100001., 100002.]))

In [63]:
dataset[0].time()

array([100001., 100002.])

In [64]:
import pandas as pd

In [65]:
dataframes = []
for traj in dataset:
    dataframes.append(pd.DataFrame.from_records(traj,columns=['x','y','z','Vx','Vy','Vz','frame','Particle']))

In [66]:
df = pd.concat(dataframes,ignore_index=True)

In [67]:
# number of trajectories or particles
len(df.groupby('Particle'))

7362

In [68]:
# number of frames
len(df.groupby('frame'))

299

In [69]:
df['Particle'] = df['Particle'].astype(np.int32)
df['frame'] = df['frame'].astype(np.int32) - 100000

In [70]:
df.reset_index(inplace=True, drop=True)
df.head()

,x,y,z,Vx,Vy,Vz,frame,Particle
0,0.016665,0.026022,-0.009375,0.000072,-0.000039,-0.000360,1,257
1,0.016737,0.025983,-0.009735,0.000000,0.000000,0.000000,2,257
2,0.014837,0.006735,0.019869,0.000144,0.000170,0.000436,1,364
3,0.014981,0.006905,0.020305,0.000000,0.000000,0.000000,2,364
4,-0.004121,-0.003949,0.013279,0.000077,0.000142,0.000191,1,187


In [71]:
df_grouped = df.reset_index().groupby('frame')
for index, group in df_grouped:
    group.to_csv(f'./output/ptv_{int(index):06d}.txt',mode='w',columns=['Particle','x','y','z','Vx','Vy','Vz'], index=False)